In [1]:
from google.colab import drive 
drive.mount('/content/drive/')
!pip install --upgrade --no-cache-dir gdown


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets
!pip install sacrebleu


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import datasets
from tqdm import tqdm
import random
import gdown
from torchvision.transforms import ToTensor, Resize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor, Resize, Compose
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
from __future__ import unicode_literals, print_function, division
import PIL 
import os
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
ferdousi_path = "/content/drive/MyDrive/ferdousi.txt"
with open(ferdousi_path, "r") as file1:
    ferdousi = file1.read()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
gdown.download(url="https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/models/glove/vectors.zip",output="vectors.zip")
!unzip "vectors.zip"

Downloading...
From: https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/models/glove/vectors.zip
To: /content/vectors.zip
100%|██████████| 48.2M/48.2M [00:00<00:00, 294MB/s]


Archive:  vectors.zip
replace vectors.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: vectors.txt             


# Preprocessing

In [7]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 0
PAD_token = 2

#initialize Lang Class
class Lang:
   def __init__(self):
       
       self.word2index = {}
       self.word2count = {}
       self.index2word = {0: "SOS", 1: "EOS", 2:"PAD"} 
       self.n_words = 3  

   def addSentence(self, sentence):
       for word in sentence.split(' '):
           self.addWord(word)

   def addWord(self, word):
       if word not in self.word2index:
           self.word2index[word] = self.n_words
           self.word2count[word] = 1
           self.index2word[self.n_words] = word
           self.n_words += 1
       else:
           self.word2count[word] += 1

In [16]:
def read_sentence(df, lang1, lang2):
   sentence1 = df[lang1]
   sentence2 = df[lang2]
   return sentence1, sentence2
   
def Word2vec(path="/content/vectors.txt"):
  file=open(path, mode="r")
  L=file.readlines(-1)
  Word_feature={}
  for i in L:
    vect=i.split()
    Word_feature[vect[0]]=np.array(vect[1:],dtype=float)
  return Word_feature

def read_file(loc, lang1, lang2):
   df = pd.read_csv(loc, delimiter='\n', header=None, names=['Beit'])[2::]
   d = {lang1:df[0::2].values[:,0].tolist() , lang2: df[1::2].values[:,0].tolist()}
   df=pd.DataFrame(d)
   return df

def process_data(lang1,lang2):
   global MAX_LENGTH
   df = read_file('/content/drive/MyDrive/ferdousi.txt', lang1, lang2)
   print("Read %s sentence pairs" % len(df))
   sentence1, sentence2 = read_sentence(df, lang1, lang2)

   source = Lang()
   target = Lang()
   pairs = []

   for i in range(len(df)):
      if len(sentence1[i].split(' '))>MAX_LENGTH:
         MAX_LENGTH = len(sentence1[i].split(' '))
     
      full = [sentence1[i], sentence2[i]]
      source.addSentence(sentence1[i])
      target.addSentence(sentence2[i])
      pairs.append(full)

   print(f"Maximum lenght of input is {MAX_LENGTH}")

   return source, target, pairs

In [17]:
lang1 = 'M1'
lang2 = 'M2'
a = read_file('/content/drive/MyDrive/ferdousi.txt', lang1, lang2)
source, target, pairs = process_data(lang1, lang2)

Read 49608 sentence pairs
Maximum lenght of input is 11


In [18]:
def indexesFromSentence(lang, sentence):
   return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
   global PAD_token
   global MAX_LENGTH
   indexes = indexesFromSentence(lang, sentence)
   pad_list=[PAD_token for i in range(MAX_LENGTH-len(indexes))]
   indexes =  indexes  + pad_list
   indexes.append(EOS_token)
   return torch.tensor(indexes, dtype=torch.long,device=device).view(-1, 1)

def tensorsFromPair(input_lang, output_lang, pair):
   input_tensor = tensorFromSentence(input_lang, pair[0])
   target_tensor = tensorFromSentence(output_lang, pair[1])
   return (input_tensor, target_tensor)

# Model

In [19]:
class Encoder(nn.Module):
   def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers,GRU=False):
       super(Encoder, self).__init__()
      
       #set the encoder input dimesion , embbed dimesion, hidden dimesion, and number of layers 
       self.input_dim = input_dim
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.num_layers = num_layers

       #initialize the embedding layer with input and embbed dimention
       self.embedding = nn.Embedding(input_dim, self.embbed_dim,padding_idx=2) #
       #intialize the GRU to take the input dimetion of embbed, and output dimention of hidden and
       #set the number of gru layers
       if GRU:
        self.recurrent = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
       else:
        self.recurrent = nn.LSTM(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
              
   def forward(self, src):
       
       embedded = self.embedding(src).view(1,1,-1)
       outputs, hidden = self.recurrent(embedded)
       return outputs, hidden
   def initHidden(self):
     return torch.zeros(1, 1, self.hidden_dim, device=device)

class Decoder(nn.Module):
   def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers,GRU=False):
       super(Decoder, self).__init__()

#set the encoder output dimension, embed dimension, hidden dimension, and number of layers 
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.output_dim = output_dim
       self.num_layers = num_layers

# initialize every layer with the appropriate dimension. For the decoder layer, it will consist of an embedding, GRU, a Linear layer and a Log softmax activation function.
       self.embedding = nn.Embedding(output_dim, self.embbed_dim ,padding_idx=2) # 
       
       if GRU:
        self.recurrent = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
       else :
        self.recurrent=nn.LSTM(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)

       self.out = nn.Linear(self.hidden_dim, output_dim)
       self.softmax = nn.LogSoftmax(dim=1)
      
   def forward(self, input, hidden):

# reshape the input to (1, batch_size)
       input = input.view(1, -1)
       embedded = F.relu(self.embedding(input))
       output, hidden = self.recurrent(embedded, hidden)       
       prediction = self.softmax(self.out(output[0]))
      
       return prediction, hidden
   def initHidden(self):
      return torch.zeros(1, 1, self.hidden_dim, device=device)

class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH,mode='GRU'):
       super().__init__()
      
#initialize the encoder and decoder
       self.encoder = encoder
       self.decoder = decoder
       self.device = device
       self.mode = mode
   def forward(self, source, target, teacher_forcing_ratio=0.5):

       input_length = source.size(0) #get the input length (number of words in sentence)
       batch_size = target.shape[1] 
       target_length = target.shape[0]
       vocab_size = self.decoder.output_dim
      
#initialize a variable to hold the predicted outputs
       outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

#encode every word in a sentence
       for i in range(input_length):
           encoder_output, encoder_hidden = self.encoder(source[i])

#use the encoder’s hidden layer as the decoder hidden
       
       if self.mode == "GRU":
         decoder_hidden = encoder_hidden.to(device)
       elif self.mode == "LSTM":
         decoder_hidden = (encoder_hidden[0].to(device),encoder_hidden[1].to(device))
       
#add a token before the first predicted word
       decoder_input = torch.tensor([SOS_token], device=device)  # SOS

#topk is used to get the top K value over a list
#predict the output word from the current target word. If we enable the teaching force,  then the #next decoder input is the next word, else, use the decoder output highest value. 

       for t in range(target_length):   
           decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
           outputs[t] = decoder_output
           teacher_force = random.random() < teacher_forcing_ratio
           topv, topi = decoder_output.topk(1)
           input = (target[t] if teacher_force else topi)
           if(teacher_force == False and input.item() == EOS_token):
               break

       return outputs

# Train

In [20]:
teacher_forcing_ratio = 0.5
metric = datasets.load_metric('sacrebleu')


def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):

   model_optimizer.zero_grad()

   input_length = input_tensor.size(0)
   loss = 0
   epoch_loss = 0
 

   output = model(input_tensor, target_tensor,teacher_forcing_ratio)

   num_iter = output.size(0)


   for ot in range(num_iter):
       loss += criterion(output[ot], target_tensor[ot])
       
   metric.add(predictions=output, references=target_tensor)
   
   loss.backward()
   model_optimizer.step()
   epoch_loss = loss.item() / num_iter

   return epoch_loss,metric

def trainModel(model, source, target, pairs, num_iteration=20000,type='GRU'):
   
   model.train()

   optimizer = torch.optim.Adam(model.parameters(), lr=0.01,weight_decay=1e-5)
 
   criterion = nn.CrossEntropyLoss()
   total_loss_iterations = 0
   final_score = 0
   training_pairs = [tensorsFromPair(source, target, random.choice(pairs))
                     for i in range(num_iteration)]
  
   for iter in tqdm(range(1, num_iteration+1)):
       training_pair = training_pairs[iter - 1]
       input_tensor = training_pair[0]
       target_tensor = training_pair[1]

       loss,metric = clacModel(model, input_tensor, target_tensor, optimizer, criterion)
       s  = metric.compute()

       final_score += s['score']
       
       total_loss_iterations += loss

       if iter % 5000 == 0:
           avarage_loss= total_loss_iterations / 5000
           avg_score = final_score /5000
           total_loss_iterations = 0
           tqdm.write(f'iter :{iter} , Loss : {avarage_loss} , score :{avg_score}')
          #  scheduler.step()
           torch.save(model.state_dict(), f'/content/drive/MyDrive/{type}_Model.pt')
          
   return model

# Test

In [21]:
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])

        decoded_words = []

        output = model(input_tensor, output_tensor)

        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1)
           
            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
    return decoded_words

def evaluateRandomly(model, source, target, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('source {}'.format(pair[0]))
        print('target {}'.format(pair[1]))
        output_words = evaluate(model, source, target, pair)
        output_sentence = ' '.join(output_words)
        print('predicted {}'.format(output_sentence))

# LSTM

In [22]:
lang1 = 'M1'
lang2 = 'M2'
source, target, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))


input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 50000


encoder = Encoder(input_size, hidden_size, embed_size, num_layers,GRU=False)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers,GRU=False)

model = Seq2Seq(encoder, decoder, device,mode='LSTM').to(device)


print(encoder)
print(decoder)

model = trainModel(model, source, target, pairs, num_iteration,type='LSTM')
evaluateRandomly(model, source, target, pairs)

Read 49608 sentence pairs
Maximum lenght of input is 11
random sentence ['یکی باد برخاست از رزمگاه', 'هوا را بپوشید گرد سپاه']
Input : 12687 Output : 13323
Encoder(
  (embedding): Embedding(12687, 256, padding_idx=2)
  (recurrent): LSTM(256, 512)
)
Decoder(
  (embedding): Embedding(13323, 256, padding_idx=2)
  (recurrent): LSTM(256, 512)
  (out): Linear(in_features=512, out_features=13323, bias=True)
  (softmax): LogSoftmax(dim=1)
)


 10%|█         | 5001/50000 [03:15<39:59, 18.76it/s]

iter :5000 , Loss : 4.979427995522815 , score :0.3447447320598481


 20%|█▉        | 9999/50000 [06:23<26:48, 24.88it/s]

iter :10000 , Loss : 6.114715136496209 , score :0.7534923378830286


...

iter :40000 , Loss : 7.725218291091918 , score :3.847343872948575


 90%|████████▉ | 44999/50000 [28:22<03:36, 23.12it/s]

iter :45000 , Loss : 7.6029853504180736 , score :4.3608444151926475


100%|██████████| 50000/50000 [31:29<00:00, 26.46it/s]


iter :50000 , Loss : 7.504036335118614 , score :4.876393700286357
source همان چون بخوردند از کاسه شیر
target توگویی بخستند هر دو به تیر
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PAD PAD
source چو این دو سرافگنده شد در نبرد
target شماساس شد بی دل و روی زرد
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PAD PAD
source بفرمود تا رستم آمد بتخت
target نشست از بر گاه زیر درخت
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PAD PAD
source بگاهی که تنها بود در نهفت
target پشیمان شود زان سخنها که گفت
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PAD PAD
source به نوروز چون برنشستی به تخت
target به نزدیک او موبد نیک بخت
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PAD PAD
source نه موبد بود شاد و نه پهلوان
target نه او در جهان شاد روشن روان
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PAD PAD
source چو فرمان دهد ما همیدون کنیم
target زمین را ز خون رود جیحون کنیم
predicted همی دشت بشتافتی بشتافتی PAD PAD PAD PAD PAD PAD PA

#GRU

In [23]:
lang1 = 'M1'
lang2 = 'M2'
source, target, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))



input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 50000


encoder = Encoder(input_size, hidden_size, embed_size, num_layers,GRU=True)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers,GRU=True)

model = Seq2Seq(encoder, decoder, device,mode='GRU').to(device)


print(encoder)
print(decoder)

model = trainModel(model, source, target, pairs, num_iteration,type='GRU')
evaluateRandomly(model, source, target, pairs)

Read 49608 sentence pairs
Maximum lenght of input is 11
random sentence ['همان روز ازان مرز لشکر براند', 'ورا بسته در پوست آنجا بماند']
Input : 12687 Output : 13323
Encoder(
  (embedding): Embedding(12687, 256, padding_idx=2)
  (recurrent): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(13323, 256, padding_idx=2)
  (recurrent): GRU(256, 512)
  (out): Linear(in_features=512, out_features=13323, bias=True)
  (softmax): LogSoftmax(dim=1)
)


 10%|▉         | 4999/50000 [03:00<30:50, 24.32it/s]

iter :5000 , Loss : 5.441672592512755 , score :0.35861098150202986


 20%|█▉        | 9997/50000 [05:59<27:40, 24.10it/s]

iter :10000 , Loss : 5.355617233530666 , score :0.7034460254127358


...

iter :40000 , Loss : 6.973181796646123 , score :3.6188872110850316


 90%|████████▉ | 44999/50000 [27:02<02:55, 28.44it/s]

iter :45000 , Loss : 7.51729236164093 , score :4.134174072441573


100%|██████████| 50000/50000 [30:05<00:00, 27.69it/s]


iter :50000 , Loss : 7.444456123701708 , score :4.648357564452722
source بدان تا روان تو روشن کند
target خرد پیش مغز تو جوشن کند
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source همی رفت گودرز با شهریار
target چو آمد بدان گلشن زرنگار
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source زمین چار فرسنگ بالای اوی
target برین هم نشانست پهنای اوی
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source بر این گونه شد سنگ در پیش من
target نبود آگه از رای کم بیش من
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source وگر خاست اندر جهان رستخیز
target که بفروختی آتش مهر تیز
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source بیاراست دانا یکی رزمگاه
target به قلب اندرون ساخته جای شاه
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source همی تاختش تا بدیشان رسید
target سر جاودان چون مر او را بدید
predicted بسی کژی کژی کژی PAD PAD PAD PAD PAD PAD PAD PAD
source دگر آنک بر گرگ بدرید پوست
target همه روم یکسر پرآواز اوست
predicted

reference: https://www.guru99.com/seq2seq-model.html